In [1]:
import pandas as pd
import warnings
import re
from nltk.corpus import stopwords, names
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
warnings.filterwarnings(action='ignore') 
stops = stopwords.words('english')
stops.append('a')
wnl = WordNetLemmatizer()
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [2]:
test.facts[1]

'Lexecon Inc. was a defendant in a class action lawsuit. Under 28 USC section 1407(a), the lawsuit was transferred for pretrial proceedings to the District of Arizona. Section 1407(a) authorizes the Judicial Panel on Multidistrict Litigation to transfer civil actions with common issues of fact "to any district for coordinated or consolidated pretrial proceedings," but provides that the Panel "shall" remand any such action to the original district "at or before the conclusion of such pretrial proceedings." After claims against it were dismissed, Lexecon brought suit against Milberg Weiss Bershad Hynes & Lerach and others (Milberg) in the class action lawsuit in the Northern District of Illinois. Ultimately, the Panel, under section 1407(a), ordered the case transferred to the District of Arizona. Afterwards, Lexecon moved for the Arizona District Court to remand the case to Illinois. Milberg filed a countermotion requesting the Arizona District Court to invoke section 1404(a) to "transf

In [3]:
reg = re.compile('[a-zA-Z]+')
for i in range(len(train)):
    first = reg.findall(train.first_party[i]) 
    second = reg.findall(train.second_party[i])
    
    tp = word_tokenize(train.facts[i])
    tp = ' '.join(tp)    
    
    fact = ' '.join(reg.findall(train.facts[i]))

    for f in first:
        if f in fact:
            fact = fact.replace(f' {f} ',' <first> ')
    for f in second:
        if f in fact:
            fact = fact.replace(f' {f} ',' <second> ')
    
    tp = fact.split()
    tp = [i.lower() for i in tp]
    tp = [wnl.lemmatize(i) for i in tp]
    for j in tp:
        if j in stops:
            tp.remove(j)

    fact = ' '.join(tp)  
    tp = word_tokenize(fact)
    fact = ' '.join(tp)
                
    train.loc[i,'facts'] = fact
            
for i in range(len(test)):
    first = reg.findall(test.first_party[i])
    second = reg.findall(test.second_party[i])
    
    tp = word_tokenize(test.facts[i])
    tp = ' '.join(tp)        

    fact = ' '.join(reg.findall(test.facts[i]))
    
#     for s in stops:
#         if s in fact:
#             fact = fact.replace(s,'')
    for f in first:
        if f in fact:
            fact = fact.replace(f' {f} ',' <first> ')
    for f in second:
        if f in fact:
            fact = fact.replace(f' {f} ',' <second> ')
    
    tp = fact.split()
    tp = [i.lower() for i in tp]
    tp = [wnl.lemmatize(i) for i in tp]

    for j in tp:
        if j in stops:
            tp.remove(j)
    
    fact = ' '.join(tp)
    tp = word_tokenize(fact)
    fact = ' '.join(tp)        
    
    test.loc[i,'facts'] = fact

In [4]:
test.facts[1]

'lexecon < second > wa defendant class action lawsuit usc section lawsuit wa transferred pretrial proceeding district arizona section authorizes judicial panel multidistrict litigation transfer civil action common issue fact district coordinated consolidated pretrial proceeding provides panel shall remand any such action original district or conclusion such pretrial proceeding claim dismissed < second > brought suit < first > < first > < first > < first > < first > < first > others < first > class action lawsuit northern district illinois ultimately panel section ordered the case transferred the district arizona afterwards < second > moved the arizona district court remand the case illinois < first > filed countermotion requesting the arizona district court invoke section a transfer the case itself trial ultimately the court assigned the case to itself < first > the court appeal affirmed it judgment'